### Re-run the whole notebook

In [1]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("sqlite:///c:\\ruby\\portlt\\db\\development.sqlite3")
conlt = engine.connect()
year = 2021
quarter = 3

In [2]:
sql = """
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s AND quarter <= %s) 
OR (year = %s-1 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2021 AND quarter <= 3) 
OR (year = 2021-1 AND quarter >= 3+1)
ORDER BY year DESC, quarter DESC


In [3]:
dfc = pd.read_sql(sql, conlt)
dfc["Counter"] = 1
dfc_grp = dfc.groupby(["name"], as_index=False).sum()
dfc_grp = dfc_grp[dfc_grp["Counter"] == 4]
dfc_grp.shape

(319, 5)

In [4]:
sql = """
SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = %s-1 AND quarter <= %s) 
OR (year = %s-2 AND quarter >= %s+1)
ORDER BY year DESC, quarter DESC"""
sql = sql % (year, quarter, year, quarter)
print(sql)


SELECT name,year,quarter,q_amt
FROM epss 
WHERE (year = 2021-1 AND quarter <= 3) 
OR (year = 2021-2 AND quarter >= 3+1)
ORDER BY year DESC, quarter DESC


In [5]:
dfp = pd.read_sql(sql, conlt)
dfp["Counter"] = 1
dfp_grp = dfp.groupby(["name"], as_index=False).sum()
dfp_grp = dfp_grp[dfp_grp["Counter"] == 4]
dfp_grp.shape

(328, 5)

In [6]:
dfm = pd.merge(dfc_grp, dfp_grp, on="name", suffixes=(["_c", "_p"]), how="inner")
dfm["inc_profit"] = dfm["q_amt_c"] - dfm["q_amt_p"]
dfm["Pct"] = dfm["inc_profit"] / abs(dfm["q_amt_p"]) * 100
dfm["year"] = year
dfm["quarter"] = "Q" + str(quarter)
df_pct = dfm[["name", "year", "quarter", "q_amt_c", "q_amt_p", "inc_profit", "Pct"]]
df_pct.shape

(313, 7)

In [7]:
sql = '''
DELETE FROM yr_profits 
WHERE year = %s AND quarter = "Q%s"'''
sql = sql % (year, quarter)
rp = conlt.execute(sql)
rp.rowcount

312

In [8]:
sql = "SELECT name, id FROM tickers"
tickers = pd.read_sql(sql, conlt)
df_ins = pd.merge(df_pct, tickers, on="name", how="inner")
rcds = df_ins.values.tolist()
len(rcds)

313

In [9]:
for rcd in rcds:
    print(rcd)

['2S', 2021, 'Q3', 788297, 319496, 468801, 146.73141447780253, 727]
['AAV', 2021, 'Q3', -6769190, -3721734, -3047456, -81.88269231492632, 732]
['ACE', 2021, 'Q3', 1392091, 1498905, -106814, -7.126135412184227, 698]
['ADVANC', 2021, 'Q3', 27223149, 27334917, -111768, -0.4088836267547474, 6]
['AEONTS', 2021, 'Q3', 3826113, 3561748, 264365, 7.422338694371415, 7]
['AH', 2021, 'Q3', 1041977, -750691, 1792668, 238.80238340409034, 9]
['AIE', 2021, 'Q3', 522478, 219664, 302814, 137.85326680748778, 720]
['AIMIRT', 2021, 'Q3', 437639, 491527, -53888, -10.963385531211916, 669]
['AIT', 2021, 'Q3', 522365, 380169, 142196, 37.40336534541217, 11]
['AJ', 2021, 'Q3', 389174, 582318, -193144, -33.168131502031535, 12]
['AKR', 2021, 'Q3', 123963, 68139, 55824, 81.92664993615992, 14]
['ALUCON', 2021, 'Q3', 725936, 665303, 60633, 9.113591852133538, 17]
['AMANAH', 2021, 'Q3', 307691, 283638, 24053, 8.480175434885313, 19]
['AMARIN', 2021, 'Q3', 367215, 147666, 219549, 148.6794522774369, 20]
['AMATA', 2021, 'Q

In [10]:
sql = """
INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?)"""
print(sql)


INSERT INTO yr_profits (name, year, quarter, latest_amt, previous_amt, inc_amt, inc_pct, ticker_id) 
VALUES (?, ?, ?, ?, ?, ?, ?, ?)


In [11]:
for rcd in rcds:
    conlt.execute(sql, rcd)

### End of loop

In [12]:
cols = ["name", "year", "quarter", "q_amt_c", "q_amt_p", "inc_profit", "Pct"]

criteria_1 = df_ins.q_amt_c > 440000
df_ins.loc[criteria_1, cols].head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
0,2S,2021,Q3,788297,319496,468801,146.731414
2,ACE,2021,Q3,1392091,1498905,-106814,-7.126135
3,ADVANC,2021,Q3,27223149,27334917,-111768,-0.408884
4,AEONTS,2021,Q3,3826113,3561748,264365,7.422339
5,AH,2021,Q3,1041977,-750691,1792668,238.802383


In [13]:
criteria_2 = df_ins.q_amt_p > 400000
df_ins.loc[criteria_2, cols].head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
2,ACE,2021,Q3,1392091,1498905,-106814,-7.126135
3,ADVANC,2021,Q3,27223149,27334917,-111768,-0.408884
4,AEONTS,2021,Q3,3826113,3561748,264365,7.422339
7,AIMIRT,2021,Q3,437639,491527,-53888,-10.963386
9,AJ,2021,Q3,389174,582318,-193144,-33.168132


In [14]:
criteria_3 = df_ins.Pct > 10
df_ins.loc[criteria_3, cols].head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
0,2S,2021,Q3,788297,319496,468801,146.731414
5,AH,2021,Q3,1041977,-750691,1792668,238.802383
6,AIE,2021,Q3,522478,219664,302814,137.853267
8,AIT,2021,Q3,522365,380169,142196,37.403365
10,AKR,2021,Q3,123963,68139,55824,81.926650


In [15]:
final_criteria = criteria_1 & criteria_2 & criteria_3
df_ins.loc[final_criteria, cols].sort_values(by=["Pct"], ascending=[False]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
244,STA,2021,Q3,19731941,4106267,15625674,380.532342
32,BCH,2021,Q3,4644042,1191940,3452102,289.620451
55,CHG,2021,Q3,2644996,757353,1887643,249.242163
168,NER,2021,Q3,1667199,561189,1106010,197.083336
201,ROJNA,2021,Q3,2184589,817176,1367413,167.333965


In [16]:
df_ins.loc[final_criteria, cols].sort_values(by=["name"], ascending=[True]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
14,AMATA,2021,Q3,1058096,957203,100893,10.540397
22,ASIAN,2021,Q3,989973,624156,365817,58.609867
23,ASK,2021,Q3,1080308,903132,177176,19.617952
30,BAY,2021,Q3,30793995,26092193,4701802,18.019957
32,BCH,2021,Q3,4644042,1191940,3452102,289.620451


In [17]:
df_ins.loc[final_criteria, cols].sort_values(by=["name"], ascending=[True]).head()

,name,year,quarter,q_amt_c,q_amt_p,inc_profit,Pct
14,AMATA,2021,Q3,1058096,957203,100893,10.540397
22,ASIAN,2021,Q3,989973,624156,365817,58.609867
23,ASK,2021,Q3,1080308,903132,177176,19.617952
30,BAY,2021,Q3,30793995,26092193,4701802,18.019957
32,BCH,2021,Q3,4644042,1191940,3452102,289.620451
